In [7]:
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import time
import pandas as pd
from datasets import load_dataset 

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "wiki_labeled")
dataset = dataset['test'].select(range(50))

Generating train split: 210000 examples [00:06, 30223.30 examples/s]
Generating test split: 45000 examples [00:01, 27787.03 examples/s]
Generating validation split: 45000 examples [00:01, 30199.97 examples/s]


In [6]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector", torchscript=True)
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector", torchscript=True)
model.to("cpu")
print()

tokenizer_config.json: 100%|██████████| 296/296 [00:00<00:00, 1.55MB/s]
tokenizer.json: 100%|██████████| 14.5M/14.5M [00:00<00:00, 19.0MB/s]
special_tokens_map.json: 100%|██████████| 96.0/96.0 [00:00<00:00, 667kB/s]
config.json: 100%|██████████| 1.02k/1.02k [00:00<00:00, 7.14MB/s]
pytorch_model.bin.index.json: 100%|██████████| 31.9k/31.9k [00:00<00:00, 112MB/s]
pytorch_model-00001-of-00002.bin: 100%|██████████| 9.91G/9.91G [04:26<00:00, 37.1MB/s]
pytorch_model-00002-of-00002.bin: 100%|██████████| 2.10G/2.10G [00:54<00:00, 38.3MB/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]

In [3]:
tokenized_texts = tokenizer(dataset['text'], padding=True, truncation=True, return_tensors="pt", max_length=512).to("cpu")
input_ids = tokenized_texts['input_ids'].to("cpu")
attention_mask = tokenized_texts['attention_mask'].to("cpu")

In [26]:
model.eval()
with torch.no_grad():
    start = time.time()
    outputs = model(input_ids, attention_mask=attention_mask)
    end = time.time()
    print("Time taken for 50 samples: ", end-start)

Time taken for 50 samples:  342.37757682800293


In [4]:
def inferenced_function(input_ids, attention_mask):
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    return outputs

traced_model = torch.jit.trace(inferenced_function, (input_ids, attention_mask))

In [ ]:
torch.jit.save(traced_model, "bloomz_base_jit.pt")

In [ ]:
result = traced_model(encoded_dataset['validation']['input_ids'])

In [3]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [4]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 12010.384816
Size (MB): 4932.560034


In [12]:
tokenized_texts = [tokenizer(x['text'], padding=True, truncation=True, return_tensors="pt", max_length=512).to("cpu") for x in dataset]
time_taken = []
model = model.to("cpu")

for text in tokenized_texts:
    start_time = time.time()
    with torch.no_grad():
        logits = model(**text)
    end_time = time.time()
    time_taken.append(end_time - start_time)
output = pd.DataFrame({"times":time_taken})
output.to_csv("quantized_inference_time.csv", index=False)
display(output.describe())

,times
count,100.000000
mean,3.609569
std,1.427363
min,1.164556
25%,2.556017
50%,3.599232
75%,4.501231
max,9.016316
